<a href="https://colab.research.google.com/github/Arifabegum1104/Arifabegum_730B13AE5F60B49B9F9404FAA5A1BB04/blob/main/VOICE_TO_NOTES_GENERATOR_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

True

In [39]:
# Uninstall wrong whisper package (to avoid AttributeError)
!pip uninstall -y whisper

# Install OpenAI Whisper directly from GitHub
!pip install git+https://github.com/openai/whisper.git

# Install other required packages
!pip install streamlit pyngrok nltk sumy

# Install ffmpeg (needed for audio processing)
!apt-get install ffmpeg -y
import nltk; nltk.download('punkt')

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-mtu4wl03
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-mtu4wl03
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [40]:
%%writefile app.py
import streamlit as st
import whisper
from datetime import datetime
import nltk
nltk.download('all')
from nltk.corpus import stopwords
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer # <-- NEW: Import Sumy's built-in Tokenizer
import os
import tempfile

# --- NLTK and Sumy Setup ---

# Download NLTK resources (only needed once, using st.cache_resource is a good practice)
@st.cache_resource
def setup_nltk():
    # Ensure both 'punkt' and 'stopwords' are downloaded
    try:
        nltk.data.find('tokenizers/punkt')
    except nltk.downloader.DownloadError:
        nltk.download('punkt', quiet=True)

    try:
        nltk.data.find('corpora/stopwords')
    except nltk.downloader.DownloadError:
        nltk.download('stopwords', quiet=True)

    return set(stopwords.words('english'))

stop_words = setup_nltk()

# --- Streamlit App Layout (No changes needed here from your working logic) ---

# Use Streamlit's resource caching to only load the model once
@st.cache_resource
def load_whisper_model(model_name):
    return whisper.load_model(model_name)

# Set wide layout and title
st.set_page_config(layout="wide", page_title="Lecture-to-Notes Generator")

st.title("💡 Smart Lecture-to-Notes System")
st.subheader("Transforming audio lectures into organized notes.")

# 1. Sidebar for File Upload and Settings
with st.sidebar:
    st.header("Upload Audio")
    uploaded_file = st.file_uploader(
        "Choose lecture audio",
        type=["wav", "mp3", "m4a", "flac"],
        help="Maximum file size is typically 200MB."
    )

    st.markdown("---")
    # Model Selector
    model_name = st.selectbox(
        "Whisper Model Quality",
        ('tiny (Fastest)', 'base (Better)', 'small (Best Quality)'),
        index=0,
        help="Higher quality models take longer to process."
    )

    # Settings for the user to control the output
    st.markdown("---")
    st.header("Settings")
    summary_sentences = st.slider(
        "Summary Length (Sentences)",
        min_value=3, max_value=15, value=5, step=1,
        help="Controls how many key sentences the summary will contain."
    )
    keyword_count = st.slider(
        "Number of Keywords",
        min_value=10, max_value=50, value=30, step=5,
        help="Controls how many of the most frequent non-stop words are shown."
    )


# 2. Main Content Area
if uploaded_file is None:
    st.info("⬆️ Please upload a lecture audio file in the sidebar to begin processing.")

if uploaded_file is not None:
    st.success("File uploaded successfully! Processing...")

    # Extract the actual model name
    actual_model = model_name.split(' ')[0]

    # Display the audio file
    st.audio(uploaded_file, format='audio/wav')

    # Use a spinner to show progress
    with st.spinner(f"Transcribing using {actual_model} model... This may take a moment."):

        transcription = ""
        summary = ""
        keywords_str = ""
        file_path = ""

        try:
            # 1. Save file temporarily using tempfile
            with tempfile.NamedTemporaryFile(delete=False, suffix=f"_{uploaded_file.name}") as tmp:
                tmp.write(uploaded_file.getbuffer())
                file_path = tmp.name

            # 2. Load Model and Transcribe Audio
            model = load_whisper_model(actual_model)
            result = model.transcribe(file_path)
            transcription_list = [segment['text'] for segment in result['segments']]
            transcription = "\n".join(transcription_list)

            # 3. Summarize with LexRank (offline)
            if transcription.strip():
                # We are using Sumy's built-in Tokenizer instead of a custom one,
                # as Sumy handles NLTK's punkt download internally, avoiding the error.
                parser = PlaintextParser.from_string(transcription, Tokenizer("english"))
                summarizer = LexRankSummarizer()
                summary_output = summarizer(parser.document, sentences_count=summary_sentences)
                summary = "\n".join([str(sentence) for sentence in summary_output])
            else:
                summary = "Transcription was empty, no summary generated."


            # 4. Extract Keywords
            words = [word.strip('.,!?"()').lower() for word in transcription.split() if word.lower() not in stop_words]
            freq = nltk.FreqDist(words)
            keywords = [word for word, count in freq.most_common(keyword_count) if word]
            keywords_str = ", ".join(keywords)

        except Exception as e:
            st.error(f"An error occurred during processing: {e}")

        finally:
            # 5. Clean up the temporary file
            if os.path.exists(file_path):
                os.remove(file_path)

    st.balloons()
    st.markdown("---")
    st.header("📝 Generated Notes")

    # Create Tabs for better organization
    tab1, tab2, tab3 = st.tabs(["Full Transcription", "Key Summary", "Keywords & Download"])

    with tab1:
        st.subheader("Full Lecture Transcription")
        st.caption(f"Generated by the **Whisper {actual_model}** model.")
        st.text_area("Transcription Text", transcription, height=500)

    with tab2:
        st.subheader(f"Lecture Summary ({summary_sentences} Key Sentences)")
        st.caption("Summary generated using the LexRank algorithm.")
        st.info(summary)

    with tab3:
        st.subheader(f"Extracted Keywords (Top {keyword_count})")
        st.markdown(f"**Keywords**: `{keywords_str}`")

        st.markdown("---")
        notes_data = f"Transcription:\n{transcription}\n\nSummary:\n{summary}\n\nKeywords:\n{keywords_str}"
        st.download_button(
            label="Download Complete Notes (TXT)",
            data=notes_data,
            file_name=f"lecture_notes_{datetime.now().strftime('%Y%m%d')}.txt",
            mime="text/plain"
        )
        st.success("Notes ready for download!")

Overwriting app.py


In [41]:
!ngrok authtoken 33T1R9u095jNZcUwsyrc8QCr3JU_8VrWX8oJF6o5Winh72aF


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [43]:
# Example ngrok/streamlit setup you used before:
import time
from pyngrok import ngrok
import subprocess

!pkill streamlit
!pkill ngrok

cmd = "streamlit run app.py --server.port 8501 --server.headless true"
process = subprocess.Popen(cmd.split())
time.sleep(10)
public_url = ngrok.connect(8501)
print(f"🚀 Streamlit app is live at: {public_url}")

🚀 Streamlit app is live at: NgrokTunnel: "https://nonparabolically-postsynaptic-tenley.ngrok-free.dev" -> "http://localhost:8501"
